![001.png](lec3.imgs/001.png)

![002.png](lec3.imgs/002.png)

![003.png](lec3.imgs/003.png)

![004.png](lec3.imgs/004.png)

![005.png](lec3.imgs/005.png)

![006.png](lec3.imgs/006.png)

![007.png](lec3.imgs/007.png)

![008.png](lec3.imgs/008.png)

![009.png](lec3.imgs/009.png)

![010.png](lec3.imgs/010.png)

![011.png](lec3.imgs/011.png)

![012.png](lec3.imgs/012.png)

![013.png](lec3.imgs/013.png)

![014.png](lec3.imgs/014.png)

![015.png](lec3.imgs/015.png)

![016.png](lec3.imgs/016.png)

![017.png](lec3.imgs/017.png)

![018.png](lec3.imgs/018.png)

![019.png](lec3.imgs/019.png)

![020.png](lec3.imgs/020.png)

![021.png](lec3.imgs/021.png)

![022.png](lec3.imgs/022.png)

![023.png](lec3.imgs/023.png)

![024.png](lec3.imgs/024.png)

![025.png](lec3.imgs/025.png)

![026.png](lec3.imgs/026.png)

![027.png](lec3.imgs/027.png)

![028.png](lec3.imgs/028.png)

![029.png](lec3.imgs/029.png)

![030.png](lec3.imgs/030.png)

![031.png](lec3.imgs/031.png)

![032.png](lec3.imgs/032.png)

![033.png](lec3.imgs/033.png)

![034.png](lec3.imgs/034.png)

![035.png](lec3.imgs/035.png)

![036.png](lec3.imgs/036.png)

![037.png](lec3.imgs/037.png)

![038.png](lec3.imgs/038.png)

![039.png](lec3.imgs/039.png)

![040.png](lec3.imgs/040.png)

![041.png](lec3.imgs/041.png)

![042.png](lec3.imgs/042.png)

![043.png](lec3.imgs/043.png)

![044.png](lec3.imgs/044.png)

![045.png](lec3.imgs/045.png)